In [1]:
using Revise
using SnpArrays
using LinearAlgebra
using Random
using LoopVectorization
using MendelIHT

┌ Info: Precompiling SnpArrays [4e780e97-f5bf-4111-9dc4-b70aaf691b06]
└ @ Base loading.jl:1317
┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1317


## No center/scale/impute

In [108]:
# x = SnpArray(SnpArrays.datadir("EUR_subset.bed"))
m = 4097
n = 1024
p = 1025
x = simulate_random_snparray(undef, m, n)

A = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, impute=false, center=false, scale=false)
B = ones(n, p)
C = zeros(m, p)
LinearAlgebra.mul!(C, A, B)

Ctrue = convert(Matrix{Float64}, x, model=ADDITIVE_MODEL) * B
@show all(C .≈ Ctrue)


M = 1024, Miter = 1, Mrem = 0
N = 1024, Niter = 1, Nrem = 0
P = 1025, Piter = 1, Prem = 1
vstep = 1024, vstep_log2 = 10
all(C .≈ Ctrue) = false


false

In [109]:
C

4097×1025 Matrix{Float64}:
 544.0  544.0  544.0  544.0  544.0  …  544.0  544.0  544.0  544.0  544.0
 484.0  484.0  484.0  484.0  484.0     484.0  484.0  484.0  484.0  484.0
 529.0  529.0  529.0  529.0  529.0     529.0  529.0  529.0  529.0  529.0
 545.0  545.0  545.0  545.0  545.0     545.0  545.0  545.0  545.0  545.0
 479.0  479.0  479.0  479.0  479.0     479.0  479.0  479.0  479.0  479.0
 493.0  493.0  493.0  493.0  493.0  …  493.0  493.0  493.0  493.0  493.0
 524.0  524.0  524.0  524.0  524.0     524.0  524.0  524.0  524.0  524.0
 537.0  537.0  537.0  537.0  537.0     537.0  537.0  537.0  537.0  537.0
 512.0  512.0  512.0  512.0  512.0     512.0  512.0  512.0  512.0  512.0
 534.0  534.0  534.0  534.0  534.0     534.0  534.0  534.0  534.0  534.0
 492.0  492.0  492.0  492.0  492.0  …  492.0  492.0  492.0  492.0  492.0
 499.0  499.0  499.0  499.0  499.0     499.0  499.0  499.0  499.0  499.0
 507.0  507.0  507.0  507.0  507.0     507.0  507.0  507.0  507.0  507.0
   ⋮                    

In [110]:
Ctrue

4097×1025 Matrix{Float64}:
 544.0  544.0  544.0  544.0  544.0  …  544.0  544.0  544.0  544.0  544.0
 484.0  484.0  484.0  484.0  484.0     484.0  484.0  484.0  484.0  484.0
 529.0  529.0  529.0  529.0  529.0     529.0  529.0  529.0  529.0  529.0
 545.0  545.0  545.0  545.0  545.0     545.0  545.0  545.0  545.0  545.0
 479.0  479.0  479.0  479.0  479.0     479.0  479.0  479.0  479.0  479.0
 493.0  493.0  493.0  493.0  493.0  …  493.0  493.0  493.0  493.0  493.0
 524.0  524.0  524.0  524.0  524.0     524.0  524.0  524.0  524.0  524.0
 537.0  537.0  537.0  537.0  537.0     537.0  537.0  537.0  537.0  537.0
 512.0  512.0  512.0  512.0  512.0     512.0  512.0  512.0  512.0  512.0
 534.0  534.0  534.0  534.0  534.0     534.0  534.0  534.0  534.0  534.0
 492.0  492.0  492.0  492.0  492.0  …  492.0  492.0  492.0  492.0  492.0
 499.0  499.0  499.0  499.0  499.0     499.0  499.0  499.0  499.0  499.0
 507.0  507.0  507.0  507.0  507.0     507.0  507.0  507.0  507.0  507.0
   ⋮                    

In [132]:
using Revise
using SnpArrays
using LinearAlgebra
using Random
using LoopVectorization
using MendelIHT
using Test

# any n between 4097 and 4099 doesn't work!
n = 4093
p = 10000
x = simulate_random_snparray(undef, n, p, min_ma=0)

A = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, impute=false, center=false, scale=false)
b = ones(p)
c = A * b
ctrue = convert(Matrix{Float64}, A) * b
@test all(c .≈ ctrue)

M = 1023, Miter = 0, Mrem = 253, rows_filled = 4093 


Test Passed

In [ ]:
# any n between 4097 and 4099 doesn't work!
n = 4093
p = 10000
x = simulate_random_snparray(undef, n, p, min_ma=0)

A = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, impute=false, center=false, scale=false)
b = ones(p)
c = A * b
ctrue = convert(Matrix{Float64}, A) * b
@test all(c .≈ ctrue)